In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [3]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [4]:
len(data)

174643

In [5]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [6]:
import json
with open('autograder/gg2013answers.json') as file:
    ground_truth = json.load(file)
file.closed

True

In [7]:
# test
true_award_winners = ground_truth['award_data']
awards_old = sorted([item[0] for item in true_award_winners.items()])

for name in awards_old:
    print(name)
    print('true winner:', true_award_winners[name]['winner'])
    print()

best animated feature film
true winner: brave

best director - motion picture
true winner: ben affleck

best foreign language film
true winner: amour

best mini-series or motion picture made for television
true winner: game change

best motion picture - comedy or musical
true winner: les miserables

best motion picture - drama
true winner: argo

best original score - motion picture
true winner: life of pi

best original song - motion picture
true winner: skyfall

best performance by an actor in a mini-series or motion picture made for television
true winner: kevin costner

best performance by an actor in a motion picture - comedy or musical
true winner: hugh jackman

best performance by an actor in a motion picture - drama
true winner: daniel day-lewis

best performance by an actor in a supporting role in a motion picture
true winner: christoph waltz

best performance by an actor in a supporting role in a series, mini-series or motion picture made for television
true winner: ed harris


In [8]:
# hard-coded awards 2013-2015
awards_old

['best animated feature film',
 'best director - motion picture',
 'best foreign language film',
 'best mini-series or motion picture made for television',
 'best motion picture - comedy or musical',
 'best motion picture - drama',
 'best original score - motion picture',
 'best original song - motion picture',
 'best performance by an actor in a mini-series or motion picture made for television',
 'best performance by an actor in a motion picture - comedy or musical',
 'best performance by an actor in a motion picture - drama',
 'best performance by an actor in a supporting role in a motion picture',
 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television',
 'best performance by an actor in a television series - comedy or musical',
 'best performance by an actor in a television series - drama',
 'best performance by an actress in a mini-series or motion picture made for television',
 'best performance by an actress in a motion

In [9]:
# references: https://en.wikipedia.org/wiki/70th_Golden_Globe_Awards
# hard-coded awards winners
winners = ['Argo', 'Les Miserables', 'Jessica Chastain', 'Daniel Day-Lewis', 'Jennifer Lawrence', 'Hugh Jackman', 
           'Anne Hathaway', 'Christoph Waltz',
           'Ben Affleck', 'Quentin Tarantino', 'Brave', 'Amour', 
           'Skyfall', 'Life of Pi', 
           'Homeland', 'Girls', 'Game Change', 
           'Julianne Moore', 'Kevin Costner', 
           'Claire Danes', 'Damian Lewis', 'Lena Dunham', 'Don Cheadle',
           'Maggie Smith', 'Ed Harris', 
           'Jodie Foster']
print(len(winners))

26


In [10]:
# hard-coded awards 2018-2019
awards = ['Best Motion Picture - Drama',
         'Best Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Motion Picture - Drama',
         'Best Performance by an Actor in a Motion Picture - Drama',
         'Best Performance by an Actress in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actor in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in any Motion Picture',
         'Best Performance by an Actor in a Supporting Role in any Motion Picture',
         'Best Director - Motion Picture',
         'Best Screenplay - Motion Picture',
         'Best Motion Picture - Animated',
         'Best Motion Picture - Foreign Language',
         'Best Original Score - Motion Picture',
         'Best Original Song - Motion Picture',
         'Best Television Series - Drama',
         'Best Television Series - Musical or Comedy',
         'Best Television Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actress in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actress in A Television Series - Drama',
         'Best Performance by an Actor in A Television Series - Drama',
         'Best Performance by an Actress in a Television Series - Musical or Comedy',
         'Best Performance by an Actor in a Television Series - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Cecil B. deMille Award']
print(len(awards))

26


In [11]:
import re

In [12]:
def reduce(line):
    # test
    # replace "television" to "tv"
    pattern = r'\btelevision\b'
    line = re.sub(pattern, 'tv', line.lower())

    # remove words "best", "performance", "motion", "picture", "limited", "language", "role", in", "a", "an", "any", "made", "for", "by", "b.", "award", and all punctuations
    pattern = r'\bbest\b|\bperformance\b|\bmotion\b|\bpicture\b|\blimited\b|\blanguage\b|\brole\b|\bin\b|\ba\b|\ban\b|\bany\b|\bmade\b|\bfor\b|\bby\b|\bb\b|\baward\b|[^\w\s]'
    return re.sub(pattern, ' ', line.lower()).split()

awards_reduced = [sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]

# test
for award in awards:
    ls = reduce(award)
    print(sorted(set(ls), key=lambda word: ls.index(word)))

['drama']
['musical', 'or', 'comedy']
['actress', 'drama']
['actor', 'drama']
['actress', 'musical', 'or', 'comedy']
['actor', 'musical', 'or', 'comedy']
['actress', 'supporting']
['actor', 'supporting']
['director']
['screenplay']
['animated']
['foreign']
['original', 'score']
['original', 'song']
['tv', 'series', 'drama']
['tv', 'series', 'musical', 'or', 'comedy']
['tv', 'series', 'or']
['actress', 'series', 'or', 'tv']
['actor', 'series', 'or', 'tv']
['actress', 'tv', 'series', 'drama']
['actor', 'tv', 'series', 'drama']
['actress', 'tv', 'series', 'musical', 'or', 'comedy']
['actor', 'tv', 'series', 'musical', 'or', 'comedy']
['actress', 'supporting', 'series', 'or', 'tv']
['actor', 'supporting', 'series', 'or', 'tv']
['cecil', 'demille']


In [13]:
# test
for award in awards_old:
    ls = reduce(award)
    print(sorted(set(ls), key=lambda word: ls.index(word)))

['animated', 'feature', 'film']
['director']
['foreign', 'film']
['mini', 'series', 'or', 'tv']
['comedy', 'or', 'musical']
['drama']
['original', 'score']
['original', 'song']
['actor', 'mini', 'series', 'or', 'tv']
['actor', 'comedy', 'or', 'musical']
['actor', 'drama']
['actor', 'supporting']
['actor', 'supporting', 'series', 'mini', 'or', 'tv']
['actor', 'tv', 'series', 'comedy', 'or', 'musical']
['actor', 'tv', 'series', 'drama']
['actress', 'mini', 'series', 'or', 'tv']
['actress', 'comedy', 'or', 'musical']
['actress', 'drama']
['actress', 'supporting']
['actress', 'supporting', 'series', 'mini', 'or', 'tv']
['actress', 'tv', 'series', 'comedy', 'or', 'musical']
['actress', 'tv', 'series', 'drama']
['screenplay']
['tv', 'series', 'comedy', 'or', 'musical']
['tv', 'series', 'drama']
['cecil', 'demille']


In [33]:
def generate_award_num_keywords_map(awards):
    awards_reduced = []
    award_num_keywords_map = {}
    for i, award in enumerate([sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]):
        if 'or' in award:
            # find number of words from "or" to the end of award name
            num_words = len(award) - 1 - award.index('or')
            award.remove('or')
            # do not count words from "or" to the end of award name
            award_num_keywords_map[i] = len(award) - num_words 
        else:
            award_num_keywords_map[i] = len(award)
        awards_reduced.append(award)
    return award_num_keywords_map, awards_reduced

In [34]:
award_num_keywords_map, awards_reduced = generate_award_num_keywords_map(awards)

pairs = sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])

In [35]:
for pair in pairs:
    print(awards_reduced[pair[0]])
    # (index, num_keywords_to_match)
    print(pair)
    if pair[1] < len(awards_reduced[pair[0]]):
        print('***')
    print

['drama']
(0, 1)
['musical', 'comedy']
(1, 1)
***
['actress', 'drama']
(2, 2)
['actor', 'drama']
(3, 2)
['actress', 'musical', 'comedy']
(4, 2)
***
['actor', 'musical', 'comedy']
(5, 2)
***
['actress', 'supporting']
(6, 2)
['actor', 'supporting']
(7, 2)
['director']
(8, 1)
['screenplay']
(9, 1)
['animated']
(10, 1)
['foreign']
(11, 1)
['original', 'score']
(12, 2)
['original', 'song']
(13, 2)
['tv', 'series', 'drama']
(14, 3)
['tv', 'series', 'musical', 'comedy']
(15, 3)
***
['tv', 'series']
(16, 2)
['actress', 'series', 'tv']
(17, 2)
***
['actor', 'series', 'tv']
(18, 2)
***
['actress', 'tv', 'series', 'drama']
(19, 4)
['actor', 'tv', 'series', 'drama']
(20, 4)
['actress', 'tv', 'series', 'musical', 'comedy']
(21, 4)
***
['actor', 'tv', 'series', 'musical', 'comedy']
(22, 4)
***
['actress', 'supporting', 'series', 'tv']
(23, 3)
***
['actor', 'supporting', 'series', 'tv']
(24, 3)
***
['cecil', 'demille']
(25, 2)


In [17]:
# test
'actress' not in awards_reduced[2]

False

In [18]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
    return re.sub(pattern, ' ', line)

In [19]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [20]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [21]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    return re.sub(pattern, ' ', line)

In [22]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

In [23]:
cleansed_data = []
for tweet in data:
    line = remove_retweet_prefix(tweet)
    line = remove_hashtag(line)
    line = remove_at(line)
    line = remove_url(line)
    line = cleanse(line)
    cleansed_data.append(line)

In [24]:
from fuzzywuzzy import fuzz

def remove_goldeb_globes(top_results, entity_freq_dict):
    golden_globes = [name for name in [pair[0] for pair in top_results] if fuzz.ratio(name.lower(), 'golden globes') > 60]
    for name in golden_globes:
        if name in entity_freq_dict:
            del entity_freq_dict[name]
    return entity_freq_dict


# we also consider dropping all lower cases examples or examples that contain digit(s), which are not names
def filter_names(pair_list, entity_freq_dict):
    filtered_results = []
    for pair in pair_list:
        string = ''.join(pair[0].split())
        if not all(char.islower() for char in string) and not any(char.isdigit() for char in string):
            filtered_results.append(pair)
        else:
            if pair[0] in entity_freq_dict:
                del entity_freq_dict[pair[0]]
    return filtered_results, entity_freq_dict


def merge_names(top_results, entity_freq_dict):
    names = [pair[0] for pair in top_results]
    names_clusters = []
    for name in names:
        # each name starts as a cluster
        cluster = [name]
        names_to_reduce = names[:]
        names_to_reduce.remove(name)
        # one vs. all comparisons
        for i in names_to_reduce:
            ratio = fuzz.ratio(name.lower(), i.lower())
            # if similarity is larger than 75 or one name is contained in the other name
            if ratio > 75 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
                cluster.append(i)
        # if multiple names are identified in one cluster
        if len(cluster) > 1:
            names_clusters.append(cluster)

    # find names clusters that should merge
    # ['Amy Poehler', 'Amy', 'Amy Poelher']
    # ['Tina', 'Tina Fey']

    # sort clusters
    names_clusters.sort()
    # sort within each cluster
    names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
    # remove overlaps
    names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
    # sort by length from shortest to longest (merge from the shortest)
    names_clusters_reduced.sort(key=len)

    # weighted frequency of an entity is defined by its frequency multiplied by its string length
    def weighted_freq(element):
        return entity_freq_dict[element] * len(element)

    e = entity_freq_dict.copy()
    for cluster in names_clusters_reduced:
        # select the entity name with highest weighted frequency
        selected_entity_name = max(cluster, key=weighted_freq)
        cluster.remove(selected_entity_name)
        # for names to be merged to the selected entity name
        for name in cluster:
            # if not deleted in previous cases, cumulate frequencies to the selected entity
            if name in e and selected_entity_name in e:
                e[selected_entity_name] += e[name]
                del e[name]

    top_10 = sorted(e.items(), key=lambda pair: pair[1], reverse=True)[:10]
    return top_10

In [25]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [26]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[entity.text]=[entity.label_]
    return tags

In [27]:
# some ideas to improve spaCy's Name Entity Recognizer
# https://github.com/explosion/spaCy/issues/1655
# https://github.com/explosion/spacy/blob/master/examples/training/train_new_entity_type.py
# https://spacy.io/usage/training#section-ner

In [28]:
# %%time
# # test
# num = 0
# pattern = re.compile(r'(\bwin)|(\bwon\b)|(\bbest\b)')
# pattern2 = re.compile(r'\bdrama\b')
# # pattern3 = re.compile(r'\bmovie\b')
# e = {}
# for line in cleansed_data:    
#     match = re.findall(pattern, line.lower())    
#     match2 = re.findall(pattern2, line.lower())  
#     match3 = re.findall(pattern3, line.lower())  
#     if match and match2:
#         tags = identify_entities(line)
#         # check the first 10 cases
#         if count < 10:
#             print(num)
#             print(match, match2)
#             print(line)
#             print(tags.keys())
#             print()
        
#         for entity in tags.keys():
#             entity = entity.strip() 
#             if len(entity) > 1:    
#                 if entity not in e:
#                     e[entity] = 1
#                 else:
#                     e[entity] += 1
#         num += 1
    
# print(count)
# print(sorted(e.items(), key=lambda pair: pair[1], reverse=True)[:100])

In [29]:
def check_target_words(awards_reduced):
    target_words = ['actor', 'actress', 'director', 'score', 'song']
    for word in target_words:
        if word in awards_reduced:
            return word
    return False

check_target_words(awards_reduced[12])

'score'

In [30]:
def find_award_winner(awards_reduced, award_index, verbose=False):
    print(award_index)
    print('Predicting for:', awards[award_index])
    print('true winner:', winners[award_index])
    award = awards_reduced[award_index]
    print('award name reduced:', award)
    num_keywords_to_match = award_num_keywords_map[award_index]
    print("num keywords to match:", num_keywords_to_match)
    
    # add word boundary '\b' to prevent grabbing examples like "showing" and "wonder"    
    pattern = re.compile(r'(\bwin)|(\bwon\b)|(\bbest\b)', re.IGNORECASE)
        
    entity_dict = {}
    
    num = 0
    flag = 0
    target_word = check_target_words(awards_reduced[award_index])
    if target_word:
        print("matching target word '{0}'".format(target_word))
    
    # if len(award) != num_keywords_to_match (awards that have 'or' options)
    # and len(award) != 2 (for the case of ['musical', 'comedy']), 
    # make sure the target word in award is matched
    if len(award) != num_keywords_to_match and len(award) != 2 or target_word:
        flag = 1
        # find target word pattern to match for sure
        if target_word:
            target_word_pattern = re.compile(r'\b{0}\b'.format(target_word), re.IGNORECASE)
        # if primary keyword is not found, match for 'tv'
        else:
            target_word_pattern = re.compile(r'\b{0}\b'.format(award.index('tv')), re.IGNORECASE)
                
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
            weight = 5 if any('win' in tup for tup in match) else 1
            match_target_word = re.findall(target_word_pattern, line.lower())
            # if line contains at least number keywords to match and pattern is found
            if len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match and match_target_word:
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity = entity.strip() 
                    if len(entity) > 1:
                        if entity not in entity_dict:
                            entity_dict[entity] = weight
                        else:
                            entity_dict[entity] += weight
                num += 1
    
    else:
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
            weight = 5 if any('win' in tup for tup in match) else 1
            # if line contains at least number keywords to match and pattern is found
            if len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match:
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(any('win' in tup for tup in match))
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity = entity.strip() 
                    if len(entity) > 1:    
                        if entity not in entity_dict:
                            entity_dict[entity] = weight
                        else:
                            entity_dict[entity] += weight
                num += 1
    
    # if no results found, recursively reducing num_keywords_to_match
    while(num == 0):
        print('no results found!!! reducing num_keywords_to_match!!!')
        num_keywords_to_match -= 1
        if num_keywords_to_match == 0:
            break
            
        if flag == 1:       
            print("matching target word '{0}'".format(target_word))
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                weight = 5 if any('win' in tup for tup in match) else 1
                match_target_word = re.findall(target_word_pattern, line.lower())
                # if line contains at least number keywords to match and pattern is found
                if len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match and match_target_word:
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity = entity.strip() 
                        if len(entity) > 1:    
                            if entity not in entity_dict:
                                entity_dict[entity] = weight
                            else:
                                entity_dict[entity] += weight
                    num += 1
        else:
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                weight = 5 if any('win' in tup for tup in match) else 1
                # if line contains at least number keywords to match and pattern is found
                if len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match:
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity = entity.strip() 
                        if len(entity) > 1:    
                            if entity not in entity_dict:
                                entity_dict[entity] = weight
                            else:
                                entity_dict[entity] += weight
                    num += 1
    print('num of matches:', num)
    return entity_dict

In [31]:
%%time
# test
entity_dict = find_award_winner(awards_reduced, 0, verbose=True)

0
Predicting for: Best Motion Picture - Drama
true winner: Argo
award name reduced: ['drama']
num keywords to match: 1
[('', '', 'best')]
False
Give me Lincoln for Best Movie Drama  Silver Linings Playbooks for Comedy Musical  Homeland for Tv drama and Girls for comedy   
dict_keys(['Lincoln for Best Movie Drama  Silver Linings Playbooks for Comedy Musical  Homeland for Tv'])

[('', '', 'best')]
False
  and the   for best supporting actor in a drama goes to  Christof Waltz 
dict_keys(['  ', ' Christof Waltz'])

[('win', '', ''), ('', '', 'best')]
True
Maggie Smith wins for best supporting actress in a drama  Go Professor McGonnagall     
dict_keys(['Maggie Smith', ' Go', 'McGonnagall'])

[('', '', 'best')]
False
Congrats to the Dowager Countess      Best Supporting Actress in a drama series  
dict_keys([])

[('win', '', ''), ('', 'won', ''), ('', '', 'best')]
True
    Is Leo Dicaprio ever going to win     he won for best actor drama 4 the departed
dict_keys(['Leo Dicaprio', '4'])

num 

# Predictions for Award Winners

In [32]:
%%time
for key in award_num_keywords_map.keys():
    entity_freq_dict = find_award_winner(awards_reduced, key, verbose=False)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
    # remove 'golden globes' from identified host names
    entity_freq_dict = remove_goldeb_globes(top_results, entity_freq_dict)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:30]
    # filter for names if necessary
    if 'actor' in awards_reduced[key] or 'actress' in awards_reduced[key] or 'director' in awards_reduced[key]:
        top_results, entity_freq_dict = filter_names(top_results, entity_freq_dict)
    print('top results:')
    print(top_results)
    print('top results after merging:')
    top_10 = merge_names(top_results, entity_freq_dict)
    print(top_10)
    print()

0
Predicting for: Best Motion Picture - Drama
true winner: Argo
award name reduced: ['drama']
num keywords to match: 1
num of matches: 810
top results:
[('Homeland', 680), ('Claire Danes', 255), ('Daniel Day Lewis', 250), ('Lincoln', 229), ('Jessica Chastain', 187), ('first', 170), ('Argo', 163), ("Les Miz'", 145), ('Lewis  Chastain', 145), ('Damien Lewis', 134), ('15', 110), ('Damian Lewis', 61), ('for', 61), ('Zero Dark Thirty', 60), ('Zero', 56), ('Kathryn Bigelow', 50), ('Latest', 50), ('Breaking', 45), ('12 year old', 45), ('A lanky', 40), ('Dark Thirty', 39), ('WINNER', 30), ('RT', 30), ('Damian', 28), ('Ben Affleck', 28), ('Lewis', 27), ('Les Miserables', 26), ('CNN', 25), ('night', 24), ('Breaking Bad', 23)]
top results after merging:
[('Homeland', 680), ('Daniel Day Lewis', 645), ('Claire Danes', 255), ('Lincoln', 229), ('Jessica Chastain', 187), ('Zero Dark Thirty', 185), ('first', 170), ('Argo', 163), ("Les Miz'", 145), ('15', 110)]

1
Predicting for: Best Motion Picture - M

num of matches: 137
top results:
[('Quentin Tarantino', 281), ('Django Unchained', 79), ('Tarantino', 34), ('Django', 28), ('WINNER', 20), ('Except', 20), ('Quentin', 12), ('QT', 11), ('tonight', 11), ('Lincoln', 6), ('quentin', 5), ('Very', 5), ('2nd', 5), ('Surprised', 5), ('Wanna', 5), ('one day', 5), ('XD', 5), ('Basterds', 5), ('3028', 5), ("Django's", 5), ('Will Lincoln', 5), ('as much as thought', 5), ('Argo', 5), ("Kerri Washington's", 5), ("Django Unchained'", 5), ('Best Screenplay', 5), ('Most Words', 5), ('Pulp Fiction', 5), ('YESSSS', 5), ('Oscars  No 2012', 5)]
top results after merging:
[('Quentin Tarantino', 332), ('Django Unchained', 117), ('WINNER', 20), ('Except', 20), ('QT', 11), ('tonight', 11), ('Will Lincoln', 11), ('Very', 5), ('2nd', 5), ('Surprised', 5)]

10
Predicting for: Best Motion Picture - Animated
true winner: Brave
award name reduced: ['animated']
num keywords to match: 1
num of matches: 147
top results:
[('Brave', 47), ('Ralph', 37), ('Wreck', 31), ('A

num of matches: 1
top results:
[('Well', 1)]
top results after merging:
[('Well', 1)]

20
Predicting for: Best Performance by an Actor in A Television Series - Drama
true winner: Damian Lewis
award name reduced: ['actor', 'tv', 'series', 'drama']
num keywords to match: 4
matching target word 'actor'
num of matches: 2
top results:
[('Damien Lewis', 1), ('Well', 1)]
top results after merging:
[('Damien Lewis', 1), ('Well', 1)]

21
Predicting for: Best Performance by an Actress in a Television Series - Musical or Comedy
true winner: Lena Dunham
award name reduced: ['actress', 'tv', 'series', 'musical', 'comedy']
num keywords to match: 4
matching target word 'actress'
num of matches: 15
top results:
[('Lena Dunham', 51), ('for Girls', 5), ('RT', 5), ('Girls', 5), ('Amy Poehler', 4), ('George Clooney', 4)]
top results after merging:
[('Lena Dunham', 51), ('for Girls', 10), ('RT', 5), ('Amy Poehler', 4), ('George Clooney', 4)]

22
Predicting for: Best Performance by an Actor in a Television 